In [25]:
# Numerical analysis
import numpy as np
import jax.numpy as jnp
from jax import random

# Bayesian inference
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import arviz as az

# Utils
from random import randrange
from sklearn.metrics import mean_squared_error

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Set pseudo random number key
prng_key = random.PRNGKey(0)

<h1> Overview </h1>

We will compare how hyperprior choice $\tau$ affects performance of the horseshoe prior by replicating an experiment done by Piironen and Vehtari in "Sparsity information and regularization in the horseshoe and other shrinkage priors".

<h1> Generate Data </h1>

$$y_i \sim \beta_i + \epsilon_i$$
$$\epsilon_i \sim N(0, \sigma^2), \;\; i = 1, ..., 400$$
Similarly to Piironen's toy example, We generate $100$ data realizations. Our true parameter vector $\beta^*$ has length $D=400$ and $p^*=20$ non-zero entries with some value $A \in$ $\{1, 2, ..., 10\}$. The total number of observations per realization is $n=400$.

In [10]:
def generate_data(dim, pstar, a, sigma, nrep):
  """
  Parameters:
    int dim: The dimension of the dataset
    int pstar: The number of true non-zero signals
    real a: The magnitude of true non-zero signals
    real sigma: Standard deviation of the noise
    nrep: The number of replications
  """
  data = np.zeros((nrep, dim))                                # Initialise
  data[:,0:(pstar-1)] = data[:,0:(pstar-1)] + a               # Add signal
  data = data + np.random.normal(0, sigma, size=(nrep, dim))  # Add noise
  
  return data

In [18]:
# Test
test = generate_data(400, 20, 10, 0.1, 100)
print(test.shape)

(100, 400)


In [ ]:
# Parameters
n_realizations = 5
n = 400 # number of observations, length of beta_star 
D = n 
p_star = 20 # number of non-zero predictors
sigma = 1 # noise parameter

# Generate data
def get_data(A):
    y = [] # outputs
    beta_stars = [] 
    for _ in range(n_realizations):
        # Create true beta vector beta_star
        beta_star = np.zeros(n)
        beta_star[range(20)] = A

        # Generate response variable y
        curr_y = beta_star + np.random.randn(len(beta_star)) * sigma

        # Store the generated data
        y.append(curr_y)
        beta_stars.append(beta_star)

    # Convert lists to numpy arrays 
    y = np.array(y)
    assert y.shape == (n_realizations, D), f'{y.shape}'
    beta_stars = np.array(beta_stars)
    return y, beta_stars

all_data = {}
A_values = [1, 2, 3, 4, 5, 6]
smaller_A_values = [4, 6]
for A in A_values:
    all_data[A] = get_data(A) #all_data[A] = (y, beta_stars) for that A

<h2> Define the Model </h2>

Horseshoe prior:
$$ y_i \sim N(\beta_i, \sigma^2) \quad \text{for } i = 1,2,\ldots,n$$ 
$$ \beta_i \sim N(0, \tau^2 \lambda_i^2) $$
$$ \lambda_i \sim C^+(0, 1)$$

Small tip for latex equations: You can do this to align the equations
$$
\begin{align*}
y_i &\sim N(\beta_i, \sigma^2) \quad \text{for } i = 1,2,\ldots,n \\
\beta_i &\sim N(0, \tau^2 \lambda_i^2) \\
\lambda_i &\sim C^+(0, 1) \\
\end{align*}
$$

We will compare two different hyperpriors for $\tau$: 

$\tau \sim C^+(0, 1)$ and $\tau = \tau_0 = \frac{p^*}{D-p^*}\sigma$.

In [ ]:
def horseshoe_linear_model(tau, y=None, sigma=1):
    '''
    Parameters:
       array y: dependent variable
       int sigma: stdev of y
    '''
    if tau == "Half-Cauchy":
       tau = numpyro.sample('tau', dist.HalfCauchy(1))
    elif tau =="Tau_0":
       tau = (p_star / (D-p_star)) * sigma
    else:
       raise ValueError
    lambdas = numpyro.sample("lambdas", dist.HalfCauchy(jnp.ones(n)))
    # slightly different parametrization for efficiency
    betas = numpyro.sample("betas", dist.Normal(0, tau*lambdas))

    kappas = numpyro.deterministic("kappas", 1 / (1 + n * tau**2 * lambdas**2))

    numpyro.sample('y', dist.Normal(betas, sigma), obs=y)

In [45]:
def hs_model(sigma=1, y=None):
  P = y.size
  tau = numpyro.sample('tau', dist.HalfCauchy(1))
  lam = numpyro.sample('lam', dist.HalfCauchy(jnp.ones(P)))
  beta = numpyro.sample('beta', dist.Normal(0, tau*lam))

  # Evaluate log-likelihood
  numpyro.sample('y', dist.Normal(beta, sigma), obs=y)

In [46]:
def rhs_fixed_model(pstar, sigma=1, y=None):
    P = y.size
    tau = (pstar / (P-pstar)) * sigma
    lam = numpyro.sample("lam", dist.HalfCauchy(jnp.ones(P)))
    beta = numpyro.sample("beta", dist.Normal(0, tau*lam))

    numpyro.sample('y', dist.Normal(beta, sigma), obs=y)

<h1> Run and Evaluate MCMC </h1>

In [47]:
y = generate_data(400, 20, 10, 1, 100)

In [61]:
def run_mcmc(model, prng_key, y, **kwargs):
  nuts = NUTS(model)
  mcmc = MCMC(nuts, num_warmup=500, num_samples=1000, num_chains=4)
  mcmc.run(prng_key, y=y, **kwargs)

  return mcmc

In [62]:
# Standard horseshoe model
fit_hs = run_mcmc(hs_model, prng_key, y[1,:])

/var/folders/fc/szc97hhx4sgg1mzfl9bt0n240000gn/T/ipykernel_8815/1936875693.py:3: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(nuts, num_warmup=500, num_samples=1000, num_chains=4)
sample: 100%|██████████| 1500/1500 [00:14<00:00, 104.82it/s, 1023 steps of size 1.27e-03. acc. prob=0.80]


In [60]:
# RHS with fixed tau
fit_rhs = run_mcmc(rhs_fixed_model, prng_key, y[1,:], pstar=20)

/var/folders/fc/szc97hhx4sgg1mzfl9bt0n240000gn/T/ipykernel_8815/1936875693.py:3: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(nuts, num_warmup=500, num_samples=1000, num_chains=4)
sample: 100%|██████████| 1500/1500 [00:13<00:00, 107.24it/s, 1023 steps of size 9.73e-04. acc. prob=0.89]


In [63]:
draws = test.get_samples()
draws['beta'].mean(axis=0)

Array([ 9.90970707e+00,  8.16509151e+00,  1.11734848e+01,  8.52251625e+00,
        1.02354412e+01,  9.70056725e+00,  9.44799423e+00,  9.94846916e+00,
        9.93171501e+00,  8.36719322e+00,  9.52340412e+00,  8.65951633e+00,
        1.08054905e+01,  8.61659813e+00,  7.86649418e+00,  9.55214310e+00,
        1.06786880e+01,  9.51243877e+00,  9.19294930e+00, -2.17114985e-02,
       -6.23188633e-03, -8.88641700e-02,  3.27619240e-02,  1.75168678e-01,
        7.05813915e-02, -1.27974108e-01,  6.04104288e-02,  2.55903378e-02,
        1.26844663e-02, -4.08631098e-03, -7.91246071e-02, -1.29003167e-01,
        8.05113986e-02, -7.25871474e-02, -1.35991186e-01, -7.46583715e-02,
        1.29787028e-01, -4.69688512e-02, -9.66205522e-02,  1.17159061e-01,
        6.76649734e-02, -1.60603952e-02, -6.23149797e-02,  1.99011758e-01,
       -3.54241058e-02,  8.37858394e-02,  1.77682236e-01,  3.64898518e-03,
        4.91233319e-02, -1.10914692e-01,  3.47084589e-02,  1.85317650e-01,
        4.08909529e-01, -

In [ ]:
def run_mcmc(tau, y):
    horseshoe_mcmc = MCMC(
        NUTS(horseshoe_linear_model),
        num_warmup= 1000,
        num_samples= 1000,
        num_chains= 4
    )
    horseshoe_mcmc.run(random.PRNGKey(2), tau=tau, y=y)
    return horseshoe_mcmc.get_samples()

beta_samples = {}
all_mcmc_runs = []
kappa_samples = []
mean_sq_errors = {"Half-Cauchy": [0]*7, "Tau_0": [0]*7}

for A, (y, beta_stars) in all_data.items():
    beta_samples[A] = {"Half-Cauchy": [], "Tau_0": []}

    samples_halfcauchy = run_mcmc("Half-Cauchy", y)
    estimated_betas_halfcauchy = samples_halfcauchy['betas'].mean(axis=0)
    mse_halfcauchy = mean_squared_error(beta_stars[0], estimated_betas_halfcauchy)
    mean_sq_errors["Half-Cauchy"][A] = mse_halfcauchy
    beta_samples[A]["Half-Cauchy"].append(estimated_betas_halfcauchy)
    kappa_samples.extend(samples_halfcauchy['kappas'])

    samples_tau0 = run_mcmc("Tau_0", y)
    estimated_betas_tau0 = samples_tau0['betas'].mean(axis=0)
    mse_tau0 = mean_squared_error(beta_stars[0], estimated_betas_tau0)
    mean_sq_errors["Tau_0"][A] = mse_tau0
    beta_samples[A]["Tau_0"].append(estimated_betas_tau0)
    kappa_samples.extend(samples_tau0['kappas'])

The <a href=https://mc-stan.org/docs/2_21/reference-manual/effective-sample-size-section.html>estimated effective sample size (ESS)</a> measures how much information is lost in MCMC due to correlation between different samples.

The <a href=https://arxiv.org/abs/1812.09384>Gelman-Rubin statistic</a> ($\hat{R}$) examines how well the chains have mixed. Common thresholds used include $1.01$, $1.05$, and $1.1$.

In [ ]:
# for mcmc in all_mcmc_runs[:5]:
#     print(az.ess(mcmc))
#     print(az.rhat(mcmc))

<h1> Results </h1>

In [ ]:
def plot_posterior_betas(beta_stars, y, estimated_betas, ax):
    ax.scatter(range(D), beta_stars, color='red', s=10) # plot actual betas
    ax.scatter(range(D), estimated_betas, color='black', s=10, alpha=0.4) # plot estimated betas
    # ax.scatter(range(len(y[i])), y[i], s=10, alpha=0.2, color='grey', marker='+') # plot y values


fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 7), layout="tight")
for ax in [ax1, ax2, ax3, ax4]: 
    ax.set_facecolor("#ECECEC") # set graph background color to grey
    ax.grid(visible=True, color="black", alpha=0.1) # add grid lines

# resetting y-axis range so that the graphs line up
ax4.set_ylim([-2, 8]) 
ax3.set_ylim([-2, 8])
ax2.set_ylim([-2.5, 6]) 
ax1.set_ylim([-2.5, 6])

d = {4: {"Half-Cauchy": ax1, "Tau_0": ax2}, 6: {"Half-Cauchy": ax3, "Tau_0": ax4}}
for A in [4, 6]:
    for tau in ["Half-Cauchy", "Tau_0"]:
        y = all_data[A][0].mean(axis=0)
        beta_stars = all_data[A][1].mean(axis=0)
        estimated_betas = beta_samples[A][tau][0]
        # print(len(beta_stars))
        # print(len(y))
        # print(len(estimated_betas))
        plot_posterior_betas(beta_stars, y, estimated_betas, d[A][tau])

# add labels to graph
ax1.set_title(r'$\tau \sim C^+(0, 1)$')
ax2.set_title(r'$\tau \sim \tau_0$')
ax1.set(ylabel=r'$A=4$')
ax3.set(ylabel=r'$A=6$', xlabel=r'Entry $i$')
ax4.set(xlabel=r'Entry $i$')


In [ ]:
plt.plot(range(len(mean_sq_errors["Half-Cauchy"])), mean_sq_errors["Half-Cauchy"])
plt.plot(range(len(mean_sq_errors["Tau_0"])), mean_sq_errors["Tau_0"])

In [ ]:
kappa_samples = np.ndarray.flatten(np.array(kappa_samples))
kappa_samples

In [ ]:
sns.kdeplot(kappa_samples, legend=None)